# Updating triangular factorization

## LU Factorization

https://johnfoster.pge.utexas.edu/numerical-methods-book/LinearAlgebra_LU.html

In [71]:
import numpy as np
import scipy.linalg
import numba

from mcf_simplex_analyzer.fractionarray import FractionArray

In [72]:
A = np.array([[5, 4, -7], [5, -8, 1], [-1, 2, -3]])
A

array([[ 5,  4, -7],
       [ 5, -8,  1],
       [-1,  2, -3]])

In [73]:
P, L, U = scipy.linalg.lu(A)
P, L, U

(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[ 1.        ,  0.        ,  0.        ],
        [ 1.        ,  1.        ,  0.        ],
        [-0.2       , -0.23333333,  1.        ]]),
 array([[  5.        ,   4.        ,  -7.        ],
        [  0.        , -12.        ,   8.        ],
        [  0.        ,   0.        ,  -2.53333333]]))

In [74]:
def plu(A):
    m, n = A.shape

    assert m == n

    P = np.arange(n, dtype=int)
    L = np.zeros((n, n))
    U = A.copy().astype("float")

    for i in range(n):
        # Determine pivot
        for k in range(i, n):
            if ~np.isclose(U[i, i], 0.0):
                break
            U[[k, k + 1]] = U[[k + 1, k]]
            P[[k, k + 1]] = P[[k + 1, k]]
        
        #if np.isclose(U[i, i], 0):
        #    k = i
        #    while i <= k < n:
        #        if not np.isclose(A[k, i], 0):
        #            break

        #        k += 1

        #    if k == n:
        #        raise ValueError("singular")

        #    P[i], P[k] = P[k], P[i]
        #    U[i, ...], U[k, ...] = U[k, ...], U[i, ...]

        L[i, i] = 1
        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, :] -= L[j, i] * U[i, :]

        print(U)

    return P, L, U

In [75]:
plu(A)

[[  5.    4.   -7. ]
 [  0.  -12.    8. ]
 [  0.    2.8  -4.4]]
[[  5.           4.          -7.        ]
 [  0.         -12.           8.        ]
 [  0.           0.          -2.53333333]]
[[  5.           4.          -7.        ]
 [  0.         -12.           8.        ]
 [  0.           0.          -2.53333333]]


(array([0, 1, 2]),
 array([[ 1.        ,  0.        ,  0.        ],
        [ 1.        ,  1.        ,  0.        ],
        [-0.2       , -0.23333333,  1.        ]]),
 array([[  5.        ,   4.        ,  -7.        ],
        [  0.        , -12.        ,   8.        ],
        [  0.        ,   0.        ,  -2.53333333]]))

In [76]:
import attr

from mcf_simplex_analyzer.fractionarray import FractionArray


@attr.s
class FractionSparseArray:
    """ Sparse one-dimensional array of fractions. """

    size = attr.ib()
    indices = attr.ib()
    data: FractionArray = attr.ib()


@attr.s
class FractionCOOMatrix:
    """ Sparse two-dimensional array of fractions. """

    shape = attr.ib()

    row = attr.ib()
    col = attr.ib()

    data: FractionArray = attr.ib()

    def nnz(self):
        return len(self.data)


@attr.s
class FractionCSCMatrix:
    """ Sparse two-dimensional array of fractions. """

    shape = attr.ib()

    indptr = attr.ib()
    indices = attr.ib()

    data: FractionArray = attr.ib()

    def nnz(self):
        return len(self.data)

In [77]:
def coo_to_csc(coo: FractionCOOMatrix, sorted=False):
    m, n = coo.shape

    row_sorted = coo.row
    col_sorted = coo.col
    data = None
    if not sorted:
        asort = np.lexsort((coo.row, coo.col))

        row_sorted = coo.row[asort]
        col_sorted = coo.col[asort]

        data = coo.data[asort]
    else:
        data = coo.data.copy()

    indptr = np.empty(n + 1, dtype=np.int64)
    indices = np.empty(coo.nnz(), dtype=np.int64)

    index = 0
    for col in range(n):         
        indptr[col] = index
        while index < len(col_sorted) and col_sorted[index] == col:
            indices[index] = row_sorted[index]
            index += 1
    indptr[n] = index

    return FractionCSCMatrix(
        shape=coo.shape,
        indptr=indptr,
        indices=indices,
        data = data
    )

def csc_to_coo(csc: FractionCSCMatrix):
    pass

In [78]:
vals = 100

rng = np.random.default_rng()

row = np.random.randint(0, 10, vals)
col = np.random.randint(0, 10, vals)

p = 0.5
rank = 0
while rank != 10:
    num = (rng.uniform(0, 1, vals) < 0.7) * rng.integers(-4, 4, vals)
    rank = np.linalg.matrix_rank(num.reshape(10, 10))

B = FractionArray.from_array(num.reshape(10, 10))

x = FractionCOOMatrix(
    shape=(10, 10), row=row, col=col, data=FractionArray(numerator=num, denominator=np.ones_like(num))
)
x

FractionCOOMatrix(shape=(10, 10), row=array([9, 1, 9, 4, 5, 3, 9, 6, 9, 3, 8, 9, 5, 2, 6, 8, 1, 8, 7, 8, 5, 9,
       4, 9, 8, 8, 3, 1, 9, 3, 2, 2, 0, 4, 5, 6, 0, 9, 0, 1, 8, 4, 2, 4,
       8, 6, 1, 7, 0, 2, 0, 4, 1, 6, 9, 2, 3, 5, 3, 5, 9, 7, 1, 6, 3, 1,
       6, 6, 9, 5, 6, 8, 2, 9, 2, 2, 7, 2, 9, 0, 2, 2, 0, 3, 4, 7, 9, 5,
       5, 2, 2, 9, 3, 9, 3, 8, 6, 3, 9, 8]), col=array([6, 4, 5, 0, 0, 6, 9, 7, 6, 4, 9, 6, 6, 5, 9, 3, 4, 8, 7, 8, 6, 3,
       0, 8, 6, 1, 9, 0, 5, 7, 3, 8, 7, 5, 5, 7, 9, 2, 5, 0, 0, 1, 6, 4,
       5, 3, 8, 9, 5, 3, 3, 4, 7, 6, 4, 5, 6, 6, 0, 4, 3, 9, 5, 9, 2, 7,
       2, 4, 6, 5, 2, 0, 1, 9, 5, 3, 0, 2, 2, 5, 3, 7, 7, 8, 8, 8, 8, 8,
       2, 6, 6, 7, 7, 0, 3, 2, 8, 5, 3, 2]), data=FractionArray(numerator=[ 0  0 -4  2  3 -1 -4 -4  0  3 -2 -1 -2  2  0  0  3  0  3 -3  2  2  2  0
  3  0  0  0 -1  3 -2  0  0  0  0  1  0 -2 -3 -4  0  0 -3  3 -1 -4  0  3
  0 -4  2  2  0  0  0  0  0  0 -3  0  0 -2 -3  0  0  1  0 -1 -4 -1 -4  0
 -2 -2 -1  0  1  1  0  0 -2  2 -1  2

In [79]:
x_csc = coo_to_csc(x)
x_csc

FractionCSCMatrix(shape=(10, 10), indptr=array([  0,  10,  13,  22,  33,  41,  55,  69,  80,  91, 100]), indices=array([1, 1, 3, 4, 4, 5, 7, 8, 8, 9, 2, 4, 8, 2, 3, 5, 6, 6, 8, 8, 9, 9,
       0, 2, 2, 2, 2, 3, 6, 8, 9, 9, 9, 1, 1, 3, 4, 4, 5, 6, 9, 0, 0, 0,
       1, 2, 2, 2, 3, 4, 5, 5, 8, 9, 9, 2, 2, 2, 3, 3, 5, 5, 5, 6, 8, 9,
       9, 9, 9, 0, 0, 1, 1, 2, 3, 3, 6, 6, 7, 9, 1, 2, 3, 4, 5, 6, 7, 8,
       8, 9, 9, 0, 3, 6, 6, 7, 7, 8, 9, 9]), data=FractionArray(numerator=[ 0 -4 -3  2  2  3  1  0  0  3 -2  0  0  1  0  0  0 -4  0  0 -2  0  2 -2
 -4  0 -2  0 -4  0  2  0 -4  0  3  3  3  2  0 -1  0 -3  0  0 -3  2  0 -1
  3  0  0 -1 -1 -4 -1 -3 -4 -2 -1  0 -2  2  0  0  3  0  0 -1 -4  0 -1  0
  1  2  3 -1 -4  1  3  3  0  0  2 -1  0 -3  0  0 -3  0  3  0  0  0  0  3
 -2 -2 -4 -2], denominator=[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [80]:
import mcf_simplex_analyzer.fractionarray as fa

def csc_to_dense(csc: FractionCSCMatrix):
    A = fa.zeros(csc.shape)
    
    for j in range(len(csc.indptr) - 1):
        start, end = csc.indptr[j], csc.indptr[j + 1]
        if end - start == 0:
            continue
        
        for k in range(start, end):
            i = csc.indices[k]
            A[i, j] = csc.data[k]
            
    return A

In [81]:
C = csc_to_dense(x_csc)
C

FractionArray(numerator=[[ 0  0  0  2  0  0  0 -1  0  0]
 [-4  0  0  0  3 -3  0  1  0  0]
 [ 0 -2  1 -2  0 -1 -2  2  0  0]
 [-3  0  0  0  3  3  0 -1  2  0]
 [ 2  0  0  0  2  0  0  0 -1  0]
 [ 3  0  0  0  0 -1  0  0  0  0]
 [ 0  0 -4 -4 -1  0  0  1 -3  0]
 [ 1  0  0  0  0  0  0  3  0 -2]
 [ 0  0  0  0  0 -1  3  0 -3 -2]
 [ 3  0  0 -4  0 -1 -4  3  3 -2]], denominator=[[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]])

In [82]:
def plu_fa(A, fast=True):
    m, n = A.shape

    assert m == n

    P = np.arange(n, dtype=int)
    L = fa.zeros((n, n))
    U = A.copy()

    for i in range(n):
        if not fast:
            for k in range(i, n):
                if U[i, i] != 0:
                    break

                U[[k, k + 1]] = U[[k + 1, k]]
                P[[k, k + 1]] = P[[k + 1, k]]
        else: 
            # Determine pivot
            k = i
            while i <= k < n:
                if U[k, i] != 0:
                    break
                
                k += 1

            if U[k, i] == 0:
                raise ValueError("Matrix is singular.")

            if k != i:
                P[i], P[k] = P[k], P[i]
                U[[i, k], i:] = U[[k, i], i:]
                L[[i, k], :i] = L[[k, i], :i]
            
        L[i, i] = 1
        nonzero = np.where(U[i :, i] != 0)[0] + i
        for j in nonzero[1:]:
            factor = U[j, i] / U[i, i]
            if False:
                print("pivot", U[i, i])
                print(factor)
                print(U[i, :])
                print(U[j, :])
                print(factor * U[i, :])
            L[j, i] = factor
            U[j, i:] -= factor * U[i, i:]
            
            

        #if np.any([ np.all(row == 0) for row in U ]):
        #    print(i)
        #    print(U)

    return P, L, U

In [83]:
TEST = FractionArray.from_array(np.array([[1, 2, 1], [1, 2, 2], [2, 1, 1]]))

P, L, U = plu_fa(TEST)
P, L, U

(array([0, 2, 1]),
 FractionArray(numerator=[[1 0 0]
  [2 1 0]
  [1 0 1]], denominator=[[1 1 1]
  [1 1 1]
  [1 1 1]]),
 FractionArray(numerator=[[ 1  2  1]
  [ 0 -3 -1]
  [ 0  0  1]], denominator=[[1 1 1]
  [1 1 1]
  [1 1 1]]))

In [84]:
def _lu_solve(lu: FractionArray, x: FractionArray):
    n = lu.shape[0]
    
    ans = fa.zeros_like(x)
    
    # Ly = b
    for i in range(n):
        ans[i] = x[i] - fa.dot(lu[i, :i], ans[:i])

    # Ux = y
    for i in range(n - 1, -1, -1):
        ans[i] -= fa.dot(lu[i, (i + 1) :], ans[(i + 1) :])
        ans[i] /= lu[i, i]

    return ans

def _lu_solve_trans(lu: FractionArray, x: FractionArray):
    n = lu.shape[0]
    
    ans = fa.zeros_like(x)
    
    # U^T y = b
    for i in range(n):
        ans[i] = x[i] - fa.dot(lu[i, :i], ans[:i])
        ans[i] /= lu[i, i]

    # L^t x = y
    for i in range(n - 1, -1, -1):
        ans[i] -= fa.dot(lu[i, (i + 1) :], ans[(i + 1) :])

    return ans

def _inverse_permutation(perm: np.ndarray):
    perm_inverse = np.empty_like(perm)
    
    perm_inverse[perm] = perm
    
    return perm_inverse

@attr.s
class PLUSolver:
    lu: FractionArray = attr.ib()
    perm: np.ndarray = attr.ib()
    _perm_trans: np.ndarray = attr.ib(default=None)

    @lu.validator
    def check_square(self, attribute, value):
        m, n = self.lu.shape

        if m != n:
            raise ValueError("matrix must be square")

    def ftran(self, b: FractionArray):
        if self._perm_trans is None:
            self._perm_trans = _inverse_permutation(self.perm)
            
        return _lu_solve(self.lu, b)[self._perm_trans]
    
    def btran(self, b : FractionArray):
        return _lu_solve_trans(self.lu.T, b)[self.perm]

In [85]:
def plu_fa_inplace(A):
    m, n = A.shape

    assert m == n

    P = np.arange(n, dtype=np.int64)

    for i in range(n):
        # Determine pivot           
        k = i
        while i <= k < n:
            if A[k, i] != 0:
                break

            k += 1

        if k == n:
            raise ValueError("Matrix is singular.")

        if k != i:
            P[[i, k]] = P[[k, i]]
            
            tmp = A[i].copy()
            A[i] = A[k]
            A[k] = tmp
        
        nonzero = np.where(A[i:, i] != 0)[0] + i
        for j in nonzero[1:]:
            factor = A[j, i] / A[i, i]
            A[j, i:] -= factor * A[i, i:]
            A[j, i] = factor
        
    return P, A

In [86]:
D = B.copy()
P, LU = plu_fa_inplace(D)
solver = PLUSolver(LU, P)
solver

PLUSolver(lu=FractionArray(numerator=[[    -2     -1     -2      2      0      0      3      0      3     -3]
 [    -1      1      0      2      3      0      3      0      2      0]
 [     0      0     -4      2      3     -1     -4     -4      0      3]
 [     1      1     -1     -3     -3      1     -8     -4     -8      1]
 [     0      0      3     -1     -4     -3     -1      4     -4     -6]
 [    -1      1      1      1      1     10     17     -2     14      4]
 [     0     -2      3     -5     -5      7     97     49   -383    -67]
 [     2      2     -1      3      1     -3   1070   -365  11870   7945]
 [     1      3     -1     11     13     89     -1    -57   1897   1447]
 [     1      4     -1     13      2      7   -608  -1094  22671 -39059]], denominator=[[   1    1    1    1    1    1    1    1    1    1]
 [   1    1    1    1    1    1    1    1    1    1]
 [   1    1    1    1    1    1    1    1    1    1]
 [   1    1    2    1    2    2    1    1    1    2]
 [   1 

In [87]:
P = B.numerator.T
r = np.ones(10, dtype=int) + 5

In [88]:
lu, piv = scipy.linalg.lu_factor(P)
scipy.linalg.lu_solve((lu, piv), r)

array([-0.92821117,  6.67766712,  0.78773138, -0.63711309, -1.84451727,
        4.72684913, -1.66916716, -0.86223406, -3.71468804,  1.91318262])

In [89]:
np.linalg.inv(P) @ r

array([-0.92821117,  6.67766712,  0.78773138, -0.63711309, -1.84451727,
        4.72684913, -1.66916716, -0.86223406, -3.71468804,  1.91318262])

In [90]:
np.linalg.solve(P, r)

array([-0.92821117,  6.67766712,  0.78773138, -0.63711309, -1.84451727,
        4.72684913, -1.66916716, -0.86223406, -3.71468804,  1.91318262])

In [91]:
LU

FractionArray(numerator=[[    -2     -1     -2      2      0      0      3      0      3     -3]
 [    -1      1      0      2      3      0      3      0      2      0]
 [     0      0     -4      2      3     -1     -4     -4      0      3]
 [     1      1     -1     -3     -3      1     -8     -4     -8      1]
 [     0      0      3     -1     -4     -3     -1      4     -4     -6]
 [    -1      1      1      1      1     10     17     -2     14      4]
 [     0     -2      3     -5     -5      7     97     49   -383    -67]
 [     2      2     -1      3      1     -3   1070   -365  11870   7945]
 [     1      3     -1     11     13     89     -1    -57   1897   1447]
 [     1      4     -1     13      2      7   -608  -1094  22671 -39059]], denominator=[[   1    1    1    1    1    1    1    1    1    1]
 [   1    1    1    1    1    1    1    1    1    1]
 [   1    1    1    1    1    1    1    1    1    1]
 [   1    1    2    1    2    2    1    1    1    2]
 [   1    1    4    

In [92]:
ans = solver.ftran(FractionArray.from_array(r))
np.array([ frac.numerator / frac.denominator for frac in ans])

array([ 1.43550526,  3.13352108, -4.17591336, -3.68759057,  5.64395402,
        2.95239509, -0.92741238,  2.58637958,  1.04601756, -3.55735682])

In [93]:
ans = solver.btran(FractionArray.from_array(r))
np.array([ frac.numerator / frac.denominator for frac in ans])

array([ 0.78773138, -0.92821117,  6.67766712, -0.63711309, -1.84451727,
        4.72684913, -1.66916716, -0.86223406, -3.71468804,  1.91318262])